In [ ]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()



In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query_sale_year = """
  SELECT 
      year(date) as sale_year,
      ROUND(AVG(price), 2) as avg_price
  FROM 
      home_sales_view
  WHERE
      bedrooms = 4
  GROUP BY 
      sale_year
  ORDER BY 
      sale_year
"""

result_sale_year = spark.sql(query_sale_year)
result_sale_year.show()



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query_year_built = """
    SELECT 
        date_built as year_built,
        ROUND(AVG(price), 2) as avg_price
    FROM 
        home_sales_view
    WHERE
        bedrooms = 3
        AND bathrooms = 3
     GROUP BY 
        year_built
     ORDER BY
        year_built
"""

result_year_built = spark.sql(query_year_built)
result_year_built.show()


In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query_ge_2000sqft = """
    SELECT 
        date_built as year_built,
        ROUND(AVG(price), 2)
    FROM 
        home_sales_view
    WHERE
        bedrooms = 3 
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
    GROUP BY 
        year_built
    ORDER BY
        year_built
"""
result_ge_2000sqft = spark.sql(query_ge_2000sqft)
result_ge_2000sqft.show()



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

query_view = """
    SELECT
      view,
      ROUND(AVG(price),2) as avg_price
    FROM
      home_sales_view
    WHERE
      price >=350000
    GROUP BY 
      view
    ORDER BY 
      view
"""
start_time = time.time()
result_view = spark.sql(query_view)
result_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales_view")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

query_view = """
    SELECT
      view,
      ROUND(AVG(price),2) as avg_price
    FROM
      home_sales_view
    WHERE
      price >=350000
    GROUP BY 
      view
    ORDER BY 
      view
"""
start_time = time.time()
result_view = spark.sql(query_view)
result_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [ ]:
# 11. Read the formatted parquet data.
df_parquet = spark.read.parquet("home_sales_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

query_view_p = """
    SELECT
      view,
      ROUND(AVG(price),2) as avg_price
    FROM
      home_sales_parquet
    WHERE
      price >=350000
    GROUP BY 
      view
    ORDER BY 
      view
"""


start_time = time.time()
result_view_p = spark.sql(query_view_p)
result_view_p.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales_view")


In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales_view")
print("Is home_sales_view cached? ", is_cached)
